In [25]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
import numpy as np
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, auc
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import RFE
import itertools
from xgboost import XGBClassifier
from tabulate import tabulate
pd.set_option('display.max_columns', None)

from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout,SimpleRNN,Input,GRU
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.utils.np_utils import to_categorical


In [49]:
df = pd.read_csv("E:/PhD-CS_Study Stuff_BDALab_PC/Nisha ciciot 2023/data/CICIoT2023/part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv")
df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0.00,1.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.00,54.00,54.000000,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,57.04,6.33,64.00,4.290556,4.290556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,581.33,54.00,66.30,54.796404,2.822973,57.04,8.292607e+07,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,441.00,42.00,42.00,42.000000,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,525.00,50.00,50.00,50.000000,0.000000,50.00,8.301570e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
4,0.117320,101.73,6.11,65.91,6.202211,6.202211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,1.01,0.04,0.00,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,644.60,57.88,131.60,67.959230,23.113111,57.88,8.297300e+07,9.5,11.346876,32.716243,3016.808286,0.19,141.55,DoS-SYN_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238682,0.000000,54.00,6.00,64.00,3.049186,3.049186,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0.00,1.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.00,54.00,54.000000,0.000000,54.00,8.334449e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
238683,0.000000,54.00,6.00,64.00,183.433732,183.433732,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00,0.00,0.00,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.00,54.00,54.000000,0.000000,54.00,8.331392e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-PSHACK_Flood
238684,0.000785,56.29,6.11,64.00,306.952216,306.952216,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.99,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.63,54.00,54.63,54.033219,0.140764,54.21,8.308883e+07,9.5,10.395538,0.200659,0.671167,0.03,141.55,DDoS-SYN_Flood
238685,0.000901,72.09,6.11,64.64,158.475986,158.475986,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.02,0.04,0.00,0.99,1.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,578.12,54.00,64.04,54.680248,2.450404,55.48,8.333177e+07,9.5,10.456522,3.475801,55.994224,0.17,141.55,DDoS-PSHACK_Flood


In [50]:
df=df[df["label"].str.contains("DoS-TCP_Flood|DoS-SYN_Flood|BenignTraffic|Mirai-greeth_flood|Mirai-udpplain|Mirai-greip_flood|DDoS-ICMP_Fragmentation|MITM-ArpSpoofing|DDoS-ACK_Fragmentation|DDoS-UDP_Fragmentation|DNS_Spoofing|Recon-HostDiscovery|Recon-OSScanRecon-PortScan|DoS-HTTP_Flood|VulnerabilityScan|DDoS-HTTP_Flood|DDoS-SlowLoris|DictionaryBruteForce|SqlInjection|BrowserHijacking|CommandInjection|Backdoor_Malware|XSS|Uploading_Attack|Recon-PingSweep") == False]
df=df.reset_index(drop=True)
df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0.00,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.00,54.000000,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,441.00,42.0,42.00,42.000000,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
2,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,525.00,50.0,50.00,50.000000,0.000000,50.00,8.301570e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
3,1.052463,108.00,6.00,64.00,1.902353,1.902353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,2.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.00,54.000000,0.000000,54.00,8.336548e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-SynonymousIP_Flood
4,0.002135,192.52,16.89,65.73,16.883240,16.883240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1885.40,156.4,180.72,177.394893,7.624944,180.72,8.300732e+07,9.5,18.809396,10.792987,328.325187,0.19,141.55,DoS-UDP_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142147,0.000000,0.58,1.05,63.78,39.611133,39.611133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,442.12,42.0,43.12,42.066817,0.264743,42.16,8.312881e+07,9.5,9.172446,0.375614,1.010427,0.07,141.55,DDoS-ICMP_Flood
142148,0.000000,54.00,6.00,64.00,3.049186,3.049186,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0.00,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.00,54.000000,0.000000,54.00,8.334449e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
142149,0.000000,54.00,6.00,64.00,183.433732,183.433732,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00,0.00,0.0,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.00,54.000000,0.000000,54.00,8.331392e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-PSHACK_Flood
142150,0.000901,72.09,6.11,64.64,158.475986,158.475986,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.02,0.04,0.0,0.99,1.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,578.12,54.0,64.04,54.680248,2.450404,55.48,8.333177e+07,9.5,10.456522,3.475801,55.994224,0.17,141.55,DDoS-PSHACK_Flood


In [24]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder



In [51]:
x=df.drop(columns=['label'])
y=df['label']

In [52]:
enc=LabelEncoder()
y=enc.fit_transform(y)

y = to_categorical(y)
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
#splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, stratify=y)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(113721, 46)
(113721, 8)
(28431, 46)
(28431, 8)


In [54]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

In [55]:
model = Sequential() # initializing model
model.add(Dense(100, input_shape=(x_train.shape[1],), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(8,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 100)               4700      
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 8)                 808       
                                                                 
Total params: 15,608
Trainable params: 15,608
Non-trainable params: 0
_________________________________________________________________


In [56]:
#callbacks: checkpoint, csv_logger
filepath="nn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csv_log=CSVLogger('nn_log.csv',separator=',',append=False)
callbacks_list=[checkpoint,csv_log]

history=model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test,y_test), callbacks=callbacks_list)

Epoch 1/50
1734/1777 [============================>.] - ETA: 0s - loss: 0.1939 - accuracy: 0.9174
Epoch 1: val_accuracy improved from -inf to 0.92371, saving model to nn.hdf5
1777/1777 [==============================] - 3s 1ms/step - loss: 0.1934 - accuracy: 0.9176 - val_loss: 0.1626 - val_accuracy: 0.9237
Epoch 2/50
1744/1777 [============================>.] - ETA: 0s - loss: 0.1463 - accuracy: 0.9333
Epoch 2: val_accuracy improved from 0.92371 to 0.94091, saving model to nn.hdf5
1777/1777 [==============================] - 2s 1ms/step - loss: 0.1460 - accuracy: 0.9333 - val_loss: 0.1457 - val_accuracy: 0.9409
Epoch 3/50
1739/1777 [============================>.] - ETA: 0s - loss: 0.1397 - accuracy: 0.9364
Epoch 3: val_accuracy improved from 0.94091 to 0.94288, saving model to nn.hdf5
1777/1777 [==============================] - 2s 1ms/step - loss: 0.1398 - accuracy: 0.9364 - val_loss: 0.1384 - val_accuracy: 0.9429
Epoch 4/50
1741/1777 [============================>.] - ETA: 0s - loss

Epoch 29/50
1764/1777 [============================>.] - ETA: 0s - loss: 0.0077 - accuracy: 0.9964
Epoch 29: val_accuracy did not improve from 0.99596
1777/1777 [==============================] - 2s 1ms/step - loss: 0.0077 - accuracy: 0.9964 - val_loss: 0.0303 - val_accuracy: 0.9953
Epoch 30/50
1760/1777 [============================>.] - ETA: 0s - loss: 0.0072 - accuracy: 0.9965
Epoch 30: val_accuracy did not improve from 0.99596
1777/1777 [==============================] - 2s 1ms/step - loss: 0.0072 - accuracy: 0.9965 - val_loss: 0.0300 - val_accuracy: 0.9931
Epoch 31/50
1771/1777 [============================>.] - ETA: 0s - loss: 0.0075 - accuracy: 0.9963
Epoch 31: val_accuracy did not improve from 0.99596
1777/1777 [==============================] - 2s 1ms/step - loss: 0.0075 - accuracy: 0.9963 - val_loss: 0.0341 - val_accuracy: 0.9958
Epoch 32/50
1771/1777 [============================>.] - ETA: 0s - loss: 0.0076 - accuracy: 0.9964
Epoch 32: val_accuracy did not improve from 0.995